In [ ]:
%run Pre-process.ipynb

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import statistics
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
usecols = ['id', 'target', 'comment_text']
train = pd.read_csv('../Data/train.csv', usecols=usecols, dtype = {'id': int})

In [ ]:
train.set_index(train['id'], inplace=True)
train = train.drop('id', axis=1).copy()

In [ ]:
train.head()

In [ ]:
#X_train = train.iloc[:2000,].comment_text
X_train = train.comment_text

#y_train = train.iloc[:2000,].target
y_train = train.target
y_train_encoded = y_train.apply(lambda x: 1 if x >= 0.5 else 0)

In [ ]:
# comments before preprocessing
for i in range(10):
    print(X_train.iloc[i])

In [ ]:
# comments after preprocessing with stopwords kept
X_train_tidy = preprocess(X_train)
for i in range(10):
    print(X_train_tidy.iloc[i])

In [ ]:
X_train_tidy = pd.DataFrame({'comment_text':X_train_tidy})
allindices = X_train_tidy.index
print('before removing dup:', X_train_tidy.shape)

X_train_tidy.drop_duplicates(subset='comment_text', keep='first', inplace=True)
print('after removing dup:', X_train_tidy.shape)

removedId = set(allindices)- set(X_train_tidy.index) # instances removed
y_train_encoded = y_train_encoded.iloc[~y_train_encoded.index.isin(removedId)]

In [ ]:
X_train_tidy = pd.Series(X_train_tidy.comment_text)

In [ ]:
# counter of number of words (tokens) in each comment
len_comments =  [len(x.split()) for x in X_train_tidy]
count_len_comments = Counter(len_comments) 

print('Number of 0-length comments:', count_len_comments[0])
print("Comments' maximum length:", max(count_len_comments))
print("Number of longest comments:", count_len_comments[max(count_len_comments)])

In [ ]:
import matplotlib.pyplot as plt
plt.hist(len_comments, bins=100)
plt.show()

In [ ]:
# remove outliers - short comments
non_zero_len = X_train_tidy.apply(lambda x: len(x.split()) != 0) # a series of booleans
X_train_remove0len = X_train_tidy[non_zero_len].copy()
non_zero_len_indices = X_train_remove0len.index
y_train_remove0len = y_train_encoded.loc[non_zero_len_indices].copy()

print(X_train_remove0len.shape)
print(y_train_remove0len.shape)

In [ ]:
# remove outliers - long comments
qt = np.quantile(len_comments, [.25,.5,.75])
iqr = qt[2] - qt[0]
outter_fence = qt[2]+3*iqr

non_long_len = X_train_remove0len.apply(lambda x: len(x.split()) <= outter_fence)
X_train_removelong = X_train_remove0len[non_long_len].copy()
non_long_len_indices = X_train_removelong.index
y_train_removelong = y_train_remove0len.loc[non_long_len_indices].copy()

print(X_train_removelong.shape)
print(y_train_removelong.shape)

In [ ]:
# output pre-processed comment
pd.DataFrame({'comment_text': X_train_removelong, 'target': y_train_removelong}).to_csv('comments_preprocessed_final.csv')